In [27]:
"""
simple Graph, wiht non-directed, non-weighted.
"""

class Graph:
    """
    can be directed 
    """
    def old(self, num_of_edges:int, edges:iter, is_directed:bool=True, is_weighted:bool=False):
        """
        @params: num_of_edges: int
        @params: is_directed: boolean
        @params: is_weighted: boolean
        @params: edges = [(from_node, to_node, weight)]
        """
        self.edges_count = num_of_edges
        self.is_directed = is_directed
        self.is_weighted = is_weighted
        # self.data is basically group by from_node
        self.data = [{} for _ in range(self.edges_count)]

        for edge in edges:
            # edge : [from_node, to_node, weight)
            if self.is_weighted:
                self.data[edge[0]][edge[1]] = edge[2]
            
    def __init__(self, num_of_edges:int, edges:iter, is_directed:bool=True, is_weighted:bool=False):
        """
        @params: num_of_edges: int
        @params: is_directed: boolean
        @params: is_weighted: boolean
        @params: edges = [(from_node, to_node, weight)]
        """
        self.edges_count = num_of_edges
        self.is_directed = is_directed
        self.is_weighted = is_weighted
        self.data = [[] for _ in range(self.edges_count)]
        if self.is_weighted:
            self.weight = [[] for _ in range(self.edges_count)]
        else:
            self.weight = []

        for edge in edges:
            # edge : [from_node, to_node, weight)
            self.data[edge[0]].append(edge[1])
            if self.is_weighted:
                self.weight[edge[0]].append(edge[2])
            # if graph is not directed
            if not self.is_directed:
                self.data[edge[1]].append(edge[0])
                if self.is_weighted:
                    self.weight[edge[1]].append(edge[2])
         
    def __repr__(self):
        s = ""
        for (i, nodes) in enumerate(self.data):
            if self.is_weighted:
                s += "{i} --> {node} \n".format(i=i, node=list(zip(nodes, self.weight[i])))
            else:
                s += "{i} --> {node} \n".format(i=i, node=nodes)
        return s
    
    def __str__(self):
        return self.__repr__()

    def draw_path_to_elem(self, parent_nodes, source, target):
        # starte from target, backtrack to source node.
        cur_node = parent_nodes[target]
        s = f"{target}"
        while cur_node is not None:
            s = f"{cur_node}-->" + s
            cur_node = parent_nodes[cur_node]
        return s

    def bfs_explorer(self, root):
        """
        for bfs we use queue data structure.
        """
        visited = [False]*self.edges_count # mapping of nodes whether discovered ?
        items = [] # bfs explrore items
        parents = [None]*self.edges_count # corresponding parents.
        weights = [None]*self.edges_count # corresponding weigths from root.
        # mark root as visited.
        queue = [root] # temp queue used to store callback nodes.
        visited[root] = True
        weights[root] = 0
        while queue:
            current_node = queue.pop(0)
            items.append(current_node)
            for edge in self.data[current_node]:
                if not visited[edge]:
                    visited[edge] = True
                    queue.append(edge)
                    parents[edge] = current_node
                    weights[edge] = weights[current_node] + 1
        # print path traveled from target to last node
        print("path_followed-->", self.draw_path_to_elem(parents, root, current_node))
        print("(node, parent_node, weight_from_root)")
        print(list(zip(items, parents, weights)))
        return items

    def dfs_explorer(self, root):
        """
        for dfs we mostly use stack.
        """
        visited = [False]*self.edges_count # mapping of nodes whether discovered ?
        items = [] # output
        stack = [root]
        while stack:
            current_node = stack.pop()
            if not visited[current_node]:
                # amrk as visited and store in items
                visited[current_node] = True
                items.append(current_node)

            # append all children to stack, which are not visited
            for edge in self.data[current_node]:
                if not visited[edge]:
                    stack.append(edge)
        return items

    def get_next_node(self, from_node, visited):
        """
        return nodes from a node which has smallest weight, which is not visited yet
        """
        weights = self.weight[from_node]
        nodes = self.data[from_node]
        min_val = weights[0]
        min_node = None
        min_weight = None
        for i, weight in enumerate(weights):
            if (min_weight is None or weight < min_weight) and not visited[nodes[i]]:
                min_weight = weight
                min_node = nodes[i]
        return min_node, min_weight
        

    def update_distances(self, cur_node, distance, parent=None):
        cur_node_distance = distance[cur_node]
        nodes = self.data[cur_node]
        weights = self.weight[cur_node]
        for node, weight in zip(nodes, weights):
            distance[node] = min(cur_node_distance + weight, distance[node])
        return distance


    def shortest_path(self, source, target):
        visited = [False] * self.edges_count
        distance = [float('inf')] * self.edges_count
        
        # update data for source node
        distance[source] = 0
        queue = [source]

        while queue and not visited[target]:
            cur_node = queue.pop(0)

            # update distances..for nodes.. (check if we found better minimum distance)
            distance = self.update_distances(cur_node, distance)

            # get next neghbours having least distance..
            next_node, _ = self.get_next_node(cur_node, visited)
            queue.append(next_node)
            
            # mark as visited.
            visited[cur_node] = True
        
        print(distance)

        return distance[target]

    
    def shortest_path_1(self, source, target):
        items = []
        visited = [False] * self.edges_count
        weights = [float('inf')] * self.edges_count
        stack = [(source, 0)]
        while stack:
            curr_node = stack.pop()
            cur_weight = weights[curr_node]
            
            items.append(curr_node)
            visited[curr_node] =  True

            nodes = self.data[curr_node]
            next_node, next_node_weight = self.get_next_node(curr_node, visited)
            if next_node is not None:
                stack.append(next_node)
        
        print(items)

In [28]:
# initialize Graph
edges1 = [(0,1),(1,2),(2,3),(3,4),(0,2),(0,4), (1,3),
 (2,4), (3,5), (4,5), (5,1), (5,2), (5,3), (5,4), (5,5)]
# create a graph
g1 = Graph(num_of_edges=6, edges=edges1)
g1

0 --> [1, 2, 4] 
1 --> [2, 3] 
2 --> [3, 4] 
3 --> [4, 5] 
4 --> [5] 
5 --> [1, 2, 3, 4, 5] 

In [9]:
"""
simple Graph, wiht non-directed, non-weighted.
"""

class WeightedGraph:
    """
    can be directed 
    """
    def __init__(self, num_of_edges:int, edges:iter, is_directed:bool=True):
        """
        @params: num_of_edges: int
        @params: is_directed: boolean
        @params: is_weighted: boolean
        @params: edges = [(from_node, to_node, weight)]
        """
        self.edges_count = num_of_edges
        self.is_directed = is_directed
        # self.data is basically group by from_node [{to_node:weight}]
        self.data = [{} for _ in range(self.edges_count)]
        for edge in edges:
            # edge : [from_node, to_node, weight)
            self.data[edge[0]][edge[1]] = edge[2]
            if self.is_directed:
                self.data[edge[1]][edge[0]] = edge[2]

    def __repr__(self):
        s = ""
        for (i, nodes) in enumerate(self.data):
            s += "{i} --> {node} \n".format(i=i, node=nodes)
        return s
    
    def __str__(self):
        return self.__repr__()

    def draw_path_to_elem(self, parent_nodes, source, target):
        # starte from target, backtrack to source node.
        cur_node = parent_nodes[target]
        s = f"{target}"
        while cur_node is not None:
            s = f"{cur_node}-->" + s
            cur_node = parent_nodes[cur_node]
        return s

    def bfs_explorer(self, root):
        """
        for bfs we use queue data structure.
        """
        visited = [False]*self.edges_count # mapping of nodes whether discovered ?
        items = [] # bfs explrore items
        parents = [None]*self.edges_count # corresponding parents.
        weights = [None]*self.edges_count # corresponding weigths from root.
        # mark root as visited.
        queue = [root] # temp queue used to store callback nodes.
        visited[root] = True
        weights[root] = 0
        while queue:
            current_node = queue.pop(0)
            items.append(current_node)
            for edge, weight in self.data[current_node].items():
                if not visited[edge]:
                    visited[edge] = True
                    queue.append(edge)
                    parents[edge] = current_node
                    weights[edge] = weights[current_node] + 1
        # print path traveled from target to last node
        print("path_followed-->", self.draw_path_to_elem(parents, root, current_node))
        print("(node, parent_node, weight_from_root)")
        print(list(zip(items, parents, weights)))
        return items

    def dfs_explorer(self, root):
        """
        for dfs we mostly use stack.
        """
        visited = [False]*self.edges_count # mapping of nodes whether discovered ?
        items = [] # output
        stack = [root]
        while stack:
            current_node = stack.pop()
            if not visited[current_node]:
                # amrk as visited and store in items
                visited[current_node] = True
                items.append(current_node)

            # append all children to stack, which are not visited
            for edge in self.data[current_node]:
                if not visited[edge]:
                    stack.append(edge)
        return items

    def get_next_node(self, from_node, visited):
        """
        return nodes from a node which has smallest weight, which is not visited yet
        """
        weights = self.weight[from_node]
        nodes = self.data[from_node]
        min_val = weights[0]
        min_node = None
        min_weight = None
        for i, weight in enumerate(weights):
            if (min_weight is None or weight < min_weight) and not visited[nodes[i]]:
                min_weight = weight
                min_node = nodes[i]
        return min_node, min_weight
        

    def update_distances(self, cur_node, distance, parent=None):
        cur_node_distance = distance[cur_node]
        nodes = self.data[cur_node]
        weights = self.weight[cur_node]
        for node, weight in zip(nodes, weights):
            distance[node] = min(cur_node_distance + weight, distance[node])
        return distance


    def shortest_path(self, source, target):
        visited = [False] * self.edges_count
        distance = [float('inf')] * self.edges_count
        
        # update data for source node
        distance[source] = 0
        queue = [source]

        while queue and not visited[target]:
            cur_node = queue.pop(0)

            # update distances..for nodes.. (check if we found better minimum distance)
            distance = self.update_distances(cur_node, distance)

            # get next neghbours having least distance..
            next_node, _ = self.get_next_node(cur_node, visited)
            queue.append(next_node)
            
            # mark as visited.
            visited[cur_node] = True
        
        print(distance)

        return distance[target]

    
    def shortest_path_1(self, source, target):
        items = []
        visited = [False] * self.edges_count
        weights = [float('inf')] * self.edges_count
        stack = [(source, 0)]
        while stack:
            curr_node = stack.pop()
            cur_weight = weights[curr_node]
            
            items.append(curr_node)
            visited[curr_node] =  True

            nodes = self.data[curr_node]
            next_node, next_node_weight = self.get_next_node(curr_node, visited)
            if next_node is not None:
                stack.append(next_node)
        
        print(items)

In [10]:
# initialize weighted graphs
edges_2= [(0,1,2),(1,2,2),(2,3,2),(3,4,2),(0,2,2),(0,4,2), (1,3,2),
 (2,4,2), (3,5,2), (4,5,2), (5,1,2), (5,2,2), (5,3,2), (5,4,2), (5,5,2)]
# create a graph
g2 = WeightedGraph(num_of_edges=6, edges=edges_2)
print(g2.data)
g2

[{1: 2, 2: 2, 4: 2}, {0: 2, 2: 2, 3: 2, 5: 2}, {1: 2, 3: 2, 0: 2, 4: 2, 5: 2}, {2: 2, 4: 2, 1: 2, 5: 2}, {3: 2, 0: 2, 2: 2, 5: 2}, {3: 2, 4: 2, 1: 2, 2: 2, 5: 2}]


0 --> {1: 2, 2: 2, 4: 2} 
1 --> {0: 2, 2: 2, 3: 2, 5: 2} 
2 --> {1: 2, 3: 2, 0: 2, 4: 2, 5: 2} 
3 --> {2: 2, 4: 2, 1: 2, 5: 2} 
4 --> {3: 2, 0: 2, 2: 2, 5: 2} 
5 --> {3: 2, 4: 2, 1: 2, 2: 2, 5: 2} 

In [11]:
# initialize Graph
edges1 = [(0,1),(1,2),(2,3),(3,4),(0,2),(0,4), (1,3),
 (2,4), (3,5), (4,5), (5,1), (5,2), (5,3), (5,4), (5,5)]
# create a graph
g1 = Graph(num_of_edges=6, edges=edges1)
print(g1.data)
g1

[{}, {}, {}, {}, {}, {}]


0 --> {} 
1 --> {} 
2 --> {} 
3 --> {} 
4 --> {} 
5 --> {} 

In [12]:
g1.bfs_explorer(0)

path_followed--> 0
(node, parent_node, weight_from_root)
[(0, None, 0)]


[0]

In [13]:
g1.dfs_explorer(0)

[0]

In [14]:
# initialize weighted graphs
edges_2= [(0,1,2),(1,2,2),(2,3,2),(3,4,2),(0,2,2),(0,4,2), (1,3,2),
 (2,4,2), (3,5,2), (4,5,2), (5,1,2), (5,2,2), (5,3,2), (5,4,2), (5,5,2)]
# create a graph
g2 = Graph(num_of_edges=6, edges=edges_2, is_weighted=True)
print(g2.data)
g2

[{1: 2, 2: 2, 4: 2}, {2: 2, 3: 2}, {3: 2, 4: 2}, {4: 2, 5: 2}, {5: 2}, {1: 2, 2: 2, 3: 2, 4: 2, 5: 2}]


AttributeError: 'Graph' object has no attribute 'weight'

In [ ]:
g2.bfs_explorer(0)

path_followed--> 0-->4-->5
(node, parent_node, weight_from_root)
[(0, None, 0), (1, 0, 1), (2, 0, 1), (4, 1, 2), (3, 0, 1), (5, 4, 2)]


[0, 1, 2, 4, 3, 5]

In [ ]:
g2.dfs_explorer(0)

[0, 4, 5, 3, 2, 1]

In [ ]:
g2.shortest_path(0,3)

[0, 2, 2, 4, 2, 6]


4